# Библиотека Pandasql

Цитата с сайта Nuancesprog: "Pandasql  —  это библиотека Python, которая позволяет обрабатывать датафреймы Pandas с помощью SQL. С точки зрения внутреннего устройства, Pandasql создает таблицу SQLite из интересующего вас датафрейма Pandas и позволяет пользователям делать запросы к таблице SQLite с помощью SQL.

Ограничения Pandasql
*   Поскольку Pandasql использует SQLite, на него распространяются все ограничения SQLite.
*   Pandasql выполняет только запросы и не может выполнять такие SQL-операции, как обновление, вставка и изменение таблиц."

##**Код ноутбука взят из вебинара "SQL запросы в Pandas", youtube-канал Karpov.Courses, ведущий Анатолий Карпов**

In [1]:
%%capture
!pip install pandasql

In [2]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [3]:
# В таблице user_actions сохраняются действия покупателей в интернет магазине, структура таблицы

# user_id    - id пользователя
# product_id - id товара
# action     - действие, просмотр товара, сохранение в корзину, покупка
# time       - время совершения события
# date       - дата события

# Для каждого дня рассчитайте, какой процент просмотров товаров завершился покупкой

In [4]:
all_user_ids = np.arange(1, 1001)
all_product_ids = np.arange(1, 101)

In [5]:
n = 10000

In [6]:
user_ids    = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_product_ids, n)

In [7]:
start_date = pd.to_datetime('2022-01-01')

In [8]:
times = pd.date_range(start_date, periods=n, freq='1min')

In [9]:
user_actions = pd.DataFrame({'user_id': user_ids, 
                             'product_id': product_ids, 
                             'time': times})

In [10]:
user_actions['action'] = 'view'

In [11]:
user_actions.head()

,user_id,product_id,time,action
0,549,37,2022-01-01 00:00:00,view
1,838,40,2022-01-01 00:01:00,view
2,316,39,2022-01-01 00:02:00,view
3,71,50,2022-01-01 00:03:00,view
4,114,78,2022-01-01 00:04:00,view


In [12]:
def generate_funnel_actions(user_id, product_id, time):
    to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_cart, 1)[0]:
        df = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(5, unit='s'), 
                           'action'    : 'add to cart'}, index=[0])
        
        if np.random.binomial(1, to_purchase, 1)[0]:
            df_purchase = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(10, unit='s'), 
                           'action'    : 'purchase'}, index=[0])
    
            df = df.append(df_purchase)
    return df

In [13]:
to_cart_df = pd.DataFrame()

In [14]:
for index, row in user_actions.iterrows():
    user_df = generate_funnel_actions(row['user_id'], row['product_id'], row['time'])
    to_cart_df = to_cart_df.append(user_df)

In [15]:
user_actions = user_actions.append(to_cart_df)
user_actions = user_actions.sort_values('time')

In [16]:
user_actions['date'] = user_actions.time.dt.date

In [17]:
user_actions.head()

,user_id,product_id,time,action,date
0,549,37,2022-01-01 00:00:00,view,2022-01-01
1,838,40,2022-01-01 00:01:00,view,2022-01-01
2,316,39,2022-01-01 00:02:00,view,2022-01-01
3,71,50,2022-01-01 00:03:00,view,2022-01-01
4,114,78,2022-01-01 00:04:00,view,2022-01-01


In [18]:
q = """SELECT date, 
              views, 
              carts, 
              purchases, 
              100 * purchases / views as purchase_percantage 
       FROM (
           SELECT date, 
            count(case when action = 'view'        then 1 else NULL end) as views, 
            count(case when action = 'add to cart' then 1 else NULL end) as carts, 
            count(case when action = 'purchase'    then 1 else NULL end) as purchases 
           FROM user_actions 
           GROUP BY date);"""

In [19]:
sqldf(q)

,date,views,carts,purchases,purchase_percantage
0,2022-01-01,1440,275,107,7
1,2022-01-02,1440,294,113,7
2,2022-01-03,1440,299,118,8
3,2022-01-04,1440,294,101,7
4,2022-01-05,1440,271,104,7
5,2022-01-06,1440,285,110,7
6,2022-01-07,1360,273,107,7
